In [1]:
import pandas as pd
import numpy as np
# import sqlalchemy
import matplotlib.pyplot as plt
import re
import sys

%matplotlib inline

In [2]:
col_desc = {
    'C150_4': 'Completion 4yr',
    'C150_L4': 'Completion < 4yr',
    'C150_4_POOLED_SUPP': 'Completion 4yr pooled suppressed',
    'C150_L4_POOLED_SUPP': 'Completion <4yr pooled suppressed',
    'C150_4_POOLED': 'Completion 4yr pooled',
    'C150_L4_POOLED': 'Completion <4yr pooled',
    'CCSIZSET': 'Carnegie classification-Size & settings',
    'CCUGPROF': 'Carnegie classification-Undergrad profile ',
    'CCBASIC': 'Carnegie classification-basic',
    'LOCALE': 'Degree of urbanization',
    'region': 'Region',
    'PREDDEG': 'Predominant degree awarded',
    'CONTROL': 'Control (public/private)',
    'RET_FT4': 'Retention 4yr',
    'RET_FTL4': 'Retention <4yr',
    'ACTCMMID': 'ACT',
    'SAT_AVG': 'SAT',
    'SAT_AVG_ALL': 'SAT all',
    'SATVRMID': 'SAT reading',
    'SATMTMID': 'SAT math',
    'SATWRMID': 'SAT writing',
    'AVGFACSAL': 'Avg faculty salary',
    'PFTFAC': 'Full time faculty rate',
    'ADM_RATE_ALL': 'Admission rate',
    'DISTANCEONLY': 'Distance only',
    'NPT4_PUB': 'Avg net price title IV institut public',
    'NPT4_PRIV': 'Avg net price title IV institut private',
    'NUM4_PUB': 'Num Title IV student, public',
    'NUM4_PRIV': 'Num Title IV student, private',
    'COSTT4_A': 'Avg cost academic year',
    'COSTT4_P': 'Avg cost program year',
    'TUITIONFEE_IN': 'In state tuition',
    'TUITIONFEE_OUT': 'Out of state tuition',
    'TUITIONFEE_PROG': 'Tuition fee program year',
    'TUITFTE': 'Net revenue per FTE student',
    'INEXPFTE': 'Expense per FTE student',
    'PCTPELL': '% Pell Grant receiver',
    'PCTFLOAN': '% Fed student loan',
    'UG25abv': '% undergrad > 25 yr',
    'PFTFTUG1_EF': 'Undergrad 1st-time degree seeking',
    'UGDS': 'Number of Undergrad degree seeking',
    'PAR_ED_PCT_1STGEN': '% 1st gen students',
    'PAR_ED_PCT_MS': '% parent education middle school',
    'PAR_ED_PCT_HS': '% parent education high school',
    'PAR_ED_PCT_PS': '% parent education post secondary',
    'DEP_INC_AVG': 'Avg income dependent stu',
    'IND_INC_AVG': 'Avg income independent stu',
    'DEBT_MDN': 'Median debt',
    'DEBT_MDN_SUPP': 'Median debt suppressed',
    'GRAD_DEBT_MDN': 'Median debt complete',
    'GRAD_DEBT_MDN_SUPP': 'Median debt completer suppressed',
    'WDRAW_DEBT_MDN': 'Median debt non-completer',
}

faculty_cols = ['AVGFACSAL', 'PFTFAC', 'ADM_RATE_ALL']
stu_score_cols = ['ACTCMMID', 'SAT_AVG', 'SAT_AVG_ALL', 'SATVRMID', 'SATMTMID', 'SATWRMID']
cost_cols = ['COSTT4_A', 'COSTT4_P', 'TUITIONFEE_IN', 'TUITIONFEE_OUT', 'TUITIONFEE_PROG', 'TUITFTE', 
             'INEXPFTE', 'NPT4_PUB', 'NPT4_PRIV']
grant_loan_cols = ['PCTPELL', 'PCTFLOAN']
stu_composition_cols = ['NUM4_PUB', 'NUM4_PRIV', 'UG25abv', 'PFTFTUG1_EF', 'UGDS', 'PAR_ED_PCT_1STGEN', 
                        'PAR_ED_PCT_MS', 'PAR_ED_PCT_HS', 
                        'PAR_ED_PCT_PS', 'DEP_INC_AVG', 'IND_INC_AVG']
debt_cols = ['DEBT_MDN', 'DEBT_MDN_SUPP', 'GRAD_DEBT_MDN', 'GRAD_DEBT_MDN_SUPP', 'WDRAW_DEBT_MDN']


In [3]:
years = ['13','12','11','10','09']
dy = {}
for y in years:
    data = pd.read_csv('MERGED20{}_PP.csv'.format(y))
    print "=== Year 20{} ===".format(y)
    print "Number of features: {}".format(len(data.columns))
    print "Number of rows: {}".format(len(data))
    dy[y] = data[sorted(col_desc.keys())]
    data = None

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (7,9,1427,1542,1561,1575,1725,1726,1727,1728) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (1427,1517,1532,1542,1545,1546,1575) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


=== Year 2013 ===
Number of features: 1729
Number of rows: 7804
=== Year 2012 ===
Number of features: 1729
Number of rows: 7793
=== Year 2011 ===
Number of features: 1729
Number of rows: 7675
=== Year 2010 ===

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (1,1408,1461,1561) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)



Number of features: 1729
Number of rows: 7414
=== Year 2009 ===
Number of features: 1729
Number of rows: 7149


In [5]:
print "Number of available data for each feature (not counting the NaN values)"
tmp = {}
for y in years:
    tmp[y] = (dy[y].isnull().sum() - len(dy[y])) * -1
print "{0:20s}{1:45s}{2:6d}{3:6d}{4:6d}{5:6d}{6:6d}".format('col', 'desc', 2013, 2012, 2011, 2010, 2009)
print "-" * 98
for k, v in tmp['13'].iteritems(): 
    print "{0:20s}{1:45s}{2:6d}{3:6d}{4:6d}{5:6d}{6:6d}".format(k, col_desc[k], v, tmp['12'][k], tmp['11'][k], 
                                                                tmp['10'][k], tmp['09'][k])

Number of available data for each feature (not counting the NaN values)
col                 desc                                           2013  2012  2011  2010  2009
--------------------------------------------------------------------------------------------------
ACTCMMID            ACT                                            1342  1328  1333  1331  1320
ADM_RATE_ALL        Admission rate                                 2484  2635  2638  2580  3027
AVGFACSAL           Avg faculty salary                             4654  4476  4579  4500  4404
C150_4              Completion 4yr                                 2448  2423  2370  2319  2269
C150_4_POOLED       Completion 4yr pooled                          2472     0     0     0     0
C150_4_POOLED_SUPP  Completion 4yr pooled suppressed               2472     0     0     0     0
C150_L4             Completion < 4yr                               3965  3923  3895  3846  3801
C150_L4_POOLED      Completion <4yr pooled                   